**Notas para contenedor de docker:**

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

```
docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_openblas -p 8888:8888 -d palmoreck/jupyterlab_openblas:1.1.0
```

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

```
docker stop jupyterlab_openblas
```

**Al ejecutar el `run` anterior se descargará la imagen y posteriormente se instalará [OpenBLAS](https://github.com/xianyi/OpenBLAS) y paquetes de Python adaptados a sus sistemas. Esto tardará $10$ minutos aproximadamente. Pueden revisar de vez en vez con el commando:**

```
docker logs jupyterlab_openblas
```

**hasta que salga un mensaje del tipo: *Successfully built scipy...The Jupyter Notebook is running at...* y poder acceder al puerto 8888 de sus máquinas**


**Nota:** no es buena idea construir una imagen de docker que ya traiga instalado `openblas` y paquetes que usemos con esta implementación. Ver por ejemplo: [docker images with architecture-optimisation](https://stackoverflow.com/questions/27919866/docker-images-with-architecture-optimisation) 

Documentación de la imagen de docker `palmoreck/jupyterlab_openblas:1.1.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/openblas).

---

Nota generada a partir de [liga](https://www.dropbox.com/s/jwu8lu4r14pb7ut/3.2.1.Sistemas_de_ecuaciones_lineales_eliminacion_Gaussiana_y_factorizacion_LU.pdf?dl=0), [liga2](https://www.dropbox.com/s/s4ch0ww1687pl76/3.2.2.Factorizaciones_matriciales_SVD_Cholesky_QR.pdf?dl=0)

# 3.2.Solución de sistemas de ecuaciones lineales y factorizaciones matriciales con OpenBLAS

En [Handle different versions of BLAS and LAPACK](https://wiki.debian.org/DebianScience/LinearAlgebraLibraries) se explica que además de ser implementaciones [BLAS: Basic Linear Algebra Subprograms](https://en.wikipedia.org/wiki/Basic_Linear_Algebra_Subprograms) y [Linear Algebra Package: LAPACK](http://www.netlib.org/lapack/explore-html/dir_fa94b7b114d387a7a8beb2e3e22bf78d.html), también son *API standard* para operaciones básicas del álgebra lineal. Muchas implementaciones de tales API's existen. Está la que vienen incluidas en las instalaciones de R o Python por ejemplo o la que se puede instalar con la línea de comando: 

```
sudo apt-get install -y libblas3 libblas-dev liblapack3 liblapack-dev
```

Ver [libblas3](https://packages.debian.org/libblas3) [libblas-dev](https://packages.debian.org/libblas-dev) [liblapack3](https://packages.debian.org/liblapack3) [liblapack-dev](https://packages.debian.org/liblapack-dev).

Sin embargo existen otras implementaciones que están optimizadas para la arquitectura de nuestras máquinas, por ejemplo:

* [OpenBLAS](https://github.com/xianyi/OpenBLAS)

* [Atlas]()

Comparación con implementación de BLAS que se encuentra integrada en `numpy`

https://wiki.debian.org/DebianScience/LinearAlgebraLibraries

In [1]:
import numpy as np

In [ ]:
np.random.seed(2020)
m=10**4
r=10**4

A=np.random.rand(m,r)
fileA='A.txt'
np.savetxt(fileA,A)

In [ ]:
np.random.seed(2021)
r=10**4
n=10**4

B=np.random.rand(r,n)
fileB='B.txt'
np.savetxt(fileB,B)

In [2]:
fileA='A.txt'
fileB='B.txt'

In [3]:
A = np.loadtxt(fileA)
B = np.loadtxt(fileB)


In [4]:
%timeit -n 1 -r 2 A@B

14.7 s ± 5.5 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [5]:
np.show_config()

blas_mkl_info:
  NOT AVAILABLE
blis_info:
  NOT AVAILABLE
openblas_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
blas_opt_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
lapack_mkl_info:
  NOT AVAILABLE
openblas_lapack_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
lapack_opt_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None)]


Comparación con OpenBlas:

In [11]:
import numpy as np

In [ ]:
np.random.seed(2020)
m=10**4
r=10**4

A=np.random.rand(m,r)
fileA='A.txt'
np.savetxt(fileA,A)

In [ ]:
np.random.seed(2021)
r=10**4
n=10**4

B=np.random.rand(r,n)
fileB='B.txt'
np.savetxt(fileB,B)

In [6]:
fileA='A.txt'
fileB='B.txt'

In [8]:
A = np.loadtxt(fileA)
B = np.loadtxt(fileB)


In [9]:
%timeit -n 1 -r 2 A@B

11.5 s ± 41.1 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [12]:
np.show_config()

blas_mkl_info:
  NOT AVAILABLE
blis_info:
  NOT AVAILABLE
openblas_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/lib/x86_64-linux-gnu']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
blas_opt_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/lib/x86_64-linux-gnu']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
lapack_mkl_info:
  NOT AVAILABLE
openblas_lapack_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/lib/x86_64-linux-gnu']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
lapack_opt_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/lib/x86_64-linux-gnu']
    language = c
    define_macros = [('HAVE_CBLAS', None)]


Not different experiment's result vs no openblas

In [ ]:
%%file mult_matrix_matrix_numpy.py
import numpy as np
m=10**4
r=10**4
n=10**4

fileA='A.txt'
fileB='B.txt'
A = np.loadtxt(fileA)
B = np.loadtxt(fileB)
C = A@B

In [1]:
%%bash
sudo perf stat -S -a --per-core -e cycles,instructions,cache-references,cache-misses -r 2 python3 mult_matrix_matrix_numpy.py


 Performance counter stats for 'system wide' (2 runs):

S0-C0           2        85516354524      cycles                                                      
S0-C0           2       122764728250      instructions              #    1.44  insn per cycle         
S0-C0           2          591263396      cache-references                                            
S0-C0           2          137764194      cache-misses              #   23.300 % of all cache refs    
S0-C1           2        84868880762      cycles                                                      
S0-C1           2       120905007085      instructions              #    1.42  insn per cycle         
S0-C1           2          582626574      cache-references                                            
S0-C1           2          143889832      cache-misses              #   24.697 % of all cache refs    
S0-C2           2        85015786453      cycles                                                      
S0-C2           

In [4]:
%%bash
python3 test.py

blas_mkl_info:
  NOT AVAILABLE
blis_info:
  NOT AVAILABLE
openblas_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/lib/x86_64-linux-gnu']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
blas_opt_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/lib/x86_64-linux-gnu']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
lapack_mkl_info:
  NOT AVAILABLE
openblas_lapack_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/lib/x86_64-linux-gnu']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
lapack_opt_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/lib/x86_64-linux-gnu']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
None


**Referencias:**

Para más sobre BLAS, LAPACK con C ver:

* [C/BLAS](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/tree/master/C/BLAS)

* [C/LAPACK](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/tree/master/C/LAPACK)

Hay implementaciones en paralelo de BLAS para sistemas de memoria distribuida. Ver por ejemplo:

* [PBLAS](http://www.netlib.org/scalapack/pblas_qref.html) y [ScaLAPACK](http://www.netlib.org/scalapack/)

También NVIDIA tiene su propia implementación de BLAS para uso con GPU's: [CUBLAS](https://docs.nvidia.com/cuda/cublas/index.html) y su implementación de LAPACK: [CUSOLVER](https://docs.nvidia.com/cuda/cusolver/index.html). Para más sobre CUBLAS y CUSOLVER ver: [C/extensiones_a_C/CUDA/CUBLAS](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/tree/master/C/extensiones_a_C/CUDA/CUBLAS) y [C/extensiones_a_C/CUDA/CUSOLVER/](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/tree/master/C/extensiones_a_C/CUDA/CUSOLVER)

Otras referencias para uso de GPU's con implementaciones de BLAS y LAPACK se encuentran:

* [MAGMA](https://icl.cs.utk.edu/magma/), [MAGMA en NVIDIA](https://developer.nvidia.com/magma), ver por ejemplo: [Matrix computations on the GPU](https://developer.nvidia.com/sites/default/files/akamai/cuda/files/Misc/mygpu.pdf)

* [NVBLAS](https://docs.nvidia.com/cuda/nvblas/)

Para otra implementación de BLAS y LAPACK ver:

* [ATLAS](http://math-atlas.sourceforge.net/atlas_install/), [Building a full LAPACK library using ATLAS and netlib's LAPACK ](http://math-atlas.sourceforge.net/atlas_install/node8.html), [ATLAS FAQ](http://math-atlas.sourceforge.net/faq.html)